In [55]:
#Cragamos librerias
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [56]:
#Cargar desde un archivo .csv sin indice
df = pd.read_csv('Mexico_Limpio.csv')
df.head(5)

,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,...,1.0,1.0,7,363,0.0,4.80,4.90,4.90,4.90,1.70
1,1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,...,1.0,1.0,365,325,64.0,4.58,4.70,4.78,4.98,0.41
2,2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,...,2.0,2.0,30,325,49.0,4.90,4.75,4.92,4.98,0.31
3,3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,...,1.0,1.6,180,257,127.0,4.91,4.96,4.98,4.96,0.83
4,4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,...,1.0,1.0,365,275,11.0,4.91,5.00,4.73,4.91,0.11


**Convertimos a numericos**

In [57]:
#df['host_response_time'] = df['host_response_time'].map({'a few days or more':1, 'within an hour':2,'within a few hours':3, 'within a day':4})
#df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})
df["host_response_rate"] = df["host_response_rate"].str.rstrip('%').astype(float) / 100
#df['room_type'] = df['room_type'].map({'Entire home/apt': 1,'Private room': 2,'Shared room': 3,'Hotel room': 4})
#df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100
df['host_verifications'] = df['host_verifications'].map({"['email', 'phone', 'work_email']":1, "['email', 'phone']":2, "['phone']": 3,"['phone', 'work_email']": 4, "['email']": 5,"[]": 6,"['email', 'work_email']": 7,"['work_email']": 8})
#df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'t': 1, 'f': 0})
#df['host_identity_verified'] = df['host_identity_verified'].map({'t': 1, 'f': 0})
#df['has_availability'] = df['has_availability'].map({'t': 1, 'f': 0})
#df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})


In [58]:
df['property_type'] = df['property_type'].replace({
    'Entire villa': 1, 'Entire home': 2, 'Entire rental unit': 3, 'Entire loft': 4,
    'Private room': 5, 'Private room in rental unit': 6, 'Private room in home': 7,
    'Private room in condo': 8, 'Private room in guest suite': 9, 'Entire condo': 10,
    'Private room in serviced apartment': 11, 'Room in serviced apartment': 12,
    'Entire guest suite': 13, 'Private room in townhouse': 14, 'Room in boutique hotel': 15,
    'Private room in bed and breakfast': 16, 'Shared room in rental unit': 17,
    'Entire serviced apartment': 18, 'Tiny home': 19, 'Entire guesthouse': 20,
    'Private room in guesthouse': 21, 'Private room in loft': 22, 'Entire bungalow': 23,
    'Casa particular': 24, 'Shared room in loft': 25, 'Entire townhouse': 26,
    'Private room in tiny home': 27, 'Shared room in guesthouse': 28, 'Room in bed and breakfast': 29,
    'Shared room in hostel': 30, 'Private room in farm stay': 31, 'Shared room in home': 32,
    'Private room in villa': 33, 'Entire cabin': 34, 'Private room in casa particular': 35,
    'Room in hotel': 36, 'Private room in floor': 37, 'Private room in cabin': 38,
    'Shared room in guest suite': 39, 'Room in aparthotel': 40, 'Entire place': 41,
    'Private room in hostel': 42, 'Earthen home': 43, 'Entire hostel': 44,
    'Private room in earthen home': 45, 'Private room in dorm': 46, 'Private room in hut': 47,
    'Entire cottage': 48, 'Entire chalet': 49, 'Shared room in dorm': 50,
    'Entire in-law': 51, 'Holiday park': 52, 'Private room in dome': 53,
    'Shared room in condo': 54, 'Campsite': 55, 'Private room in vacation home': 56,
    'Room in casa particular': 57, 'Castle': 58, 'Private room in barn': 59,
    'Shared room in hotel': 60, 'Farm stay': 61, 'Shared room in tiny home': 62,
    'Hut': 63, 'Shared room in cabin': 64, 'Private room in cottage': 65,
    'Entire home/apt': 66, 'Room in hostel': 67, 'Shared room in serviced apartment': 68,
    'Entire vacation home': 69, 'Shared room in tent': 70, 'Shared room in bed and breakfast': 71,
    'Private room in shipping container': 72, 'Private room in nature lodge': 73,
    'Private room in tower': 74, 'Private room in lighthouse': 75,
    'Shared room in casa particular': 76, 'Shared room in townhouse': 77,
    'Shared room in farm stay': 78, 'Shared room in boutique hotel': 79,
    'Shipping container': 80, 'Private room in pension': 81, 'Private room in bungalow': 82,
    'Dome': 83, 'Tent': 84, 'Private room in chalet': 85, 'Private room in tent': 86,
    'Private room in castle': 87, 'Private room in houseboat': 88, 'Boat': 89,
    'Private room in resort': 90, 'Private room in minsu': 91, 'Tower': 92
})


C:\Users\ady_b\AppData\Local\Temp\ipykernel_20808\2798425177.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['property_type'] = df['property_type'].replace({


**Reemplazar valores**

_host_acceptance_rate_

In [59]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_acceptance_rate'])
unico

array(['0%', '0.9', '10%', '100%', '11%', '13%', '14%', '15%', '17%',
       '18%', '19%', '2%', '20%', '21%', '22%', '23%', '24%', '25%',
       '26%', '27%', '28%', '29%', '3%', '30%', '31%', '32%', '33%',
       '34%', '35%', '36%', '37%', '38%', '39%', '4%', '40%', '41%',
       '42%', '43%', '44%', '45%', '46%', '47%', '48%', '49%', '5%',
       '50%', '51%', '52%', '53%', '54%', '55%', '56%', '57%', '58%',
       '59%', '6%', '60%', '61%', '62%', '63%', '64%', '65%', '66%',
       '67%', '68%', '69%', '7%', '70%', '71%', '72%', '73%', '74%',
       '75%', '76%', '77%', '78%', '79%', '8%', '80%', '81%', '82%',
       '83%', '84%', '85%', '86%', '87%', '88%', '89%', '9%', '90%',
       '91%', '92%', '93%', '94%', '95%', '96%', '97%', '98%', '99%'],
      dtype=object)

In [60]:
df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100

In [61]:
#Convierto una variable a dicotómica
df['host_acceptance_rate'] = df['host_acceptance_rate'].apply(
    lambda x: 'aceptacion_alta' if isinstance(x, (int, float)) and x > 0.80 else ('aceptacion_baja' if isinstance(x, (int, float)) and x <= 0.80 else 'Sin respuesta')
)

df

,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,...,1.0,1.0,7,363,0.0,4.80,4.90,4.90,4.90,1.70
1,1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,...,1.0,1.0,365,325,64.0,4.58,4.70,4.78,4.98,0.41
2,2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,...,2.0,2.0,30,325,49.0,4.90,4.75,4.92,4.98,0.31
3,3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,...,1.0,1.6,180,257,127.0,4.91,4.96,4.98,4.96,0.83
4,4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,...,1.0,1.0,365,275,11.0,4.91,5.00,4.73,4.91,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26577,26577,26577,https://www.airbnb.com/rooms/1252141718111439700,2024-09-25,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,"Welcome to Lomas de Bezares, a neighborhood kn...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/67472234,HostPal,...,2.0,1.0,45,365,0.0,4.80,4.90,4.90,4.90,1.70
26578,26578,26578,https://www.airbnb.com/rooms/1252159821531694005,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70
26579,26579,26579,https://www.airbnb.com/rooms/1252185591721328995,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70
26580,26580,26580,https://www.airbnb.com/rooms/1252218347521498445,2024-09-25,city scrape,Suite Queen | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70


_room_type_

In [62]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
unico

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [63]:
#Convierto una variable a dicotómica
df['room_type'] = df['room_type'].replace(["Shared room", "Private room"], "Alojamiento compartido")
df['room_type'] = df['room_type'].replace(["Entire home/apt", "Hotel room"], "Alojamiento privado")
df

,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,...,1.0,1.0,7,363,0.0,4.80,4.90,4.90,4.90,1.70
1,1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,...,1.0,1.0,365,325,64.0,4.58,4.70,4.78,4.98,0.41
2,2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,...,2.0,2.0,30,325,49.0,4.90,4.75,4.92,4.98,0.31
3,3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,...,1.0,1.6,180,257,127.0,4.91,4.96,4.98,4.96,0.83
4,4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,...,1.0,1.0,365,275,11.0,4.91,5.00,4.73,4.91,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26577,26577,26577,https://www.airbnb.com/rooms/1252141718111439700,2024-09-25,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,"Welcome to Lomas de Bezares, a neighborhood kn...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/67472234,HostPal,...,2.0,1.0,45,365,0.0,4.80,4.90,4.90,4.90,1.70
26578,26578,26578,https://www.airbnb.com/rooms/1252159821531694005,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70
26579,26579,26579,https://www.airbnb.com/rooms/1252185591721328995,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70
26580,26580,26580,https://www.airbnb.com/rooms/1252218347521498445,2024-09-25,city scrape,Suite Queen | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70


_availability_365_

In [64]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['availability_365'])
unico

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [65]:
#Convierto una variable a dicotómica
df['availability_365'] = df['availability_365'].apply(
    lambda x: 'disponible todo el año' if x == 365 else ('no disponible todo el año' if isinstance(x, (int, float)) and x < 365 else 'Sin respuesta')
)

df

,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,...,1.0,1.0,7,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
1,1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,...,1.0,1.0,365,no disponible todo el año,64.0,4.58,4.70,4.78,4.98,0.41
2,2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,...,2.0,2.0,30,no disponible todo el año,49.0,4.90,4.75,4.92,4.98,0.31
3,3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,...,1.0,1.6,180,no disponible todo el año,127.0,4.91,4.96,4.98,4.96,0.83
4,4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,...,1.0,1.0,365,no disponible todo el año,11.0,4.91,5.00,4.73,4.91,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26577,26577,26577,https://www.airbnb.com/rooms/1252141718111439700,2024-09-25,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,"Welcome to Lomas de Bezares, a neighborhood kn...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/67472234,HostPal,...,2.0,1.0,45,disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
26578,26578,26578,https://www.airbnb.com/rooms/1252159821531694005,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
26579,26579,26579,https://www.airbnb.com/rooms/1252185591721328995,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
26580,26580,26580,https://www.airbnb.com/rooms/1252218347521498445,2024-09-25,city scrape,Suite Queen | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70


_host_response_time_

In [66]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_response_time'])
unico

array(['a few days or more', 'within a day', 'within a few hours',
       'within an hour'], dtype=object)

In [67]:
df['host_response_time']=df['host_response_time'].replace(["within a few hours", "within a day", "a few days or more"], "within more than an hour")
df

,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,...,1.0,1.0,7,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
1,1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,...,1.0,1.0,365,no disponible todo el año,64.0,4.58,4.70,4.78,4.98,0.41
2,2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,...,2.0,2.0,30,no disponible todo el año,49.0,4.90,4.75,4.92,4.98,0.31
3,3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,...,1.0,1.6,180,no disponible todo el año,127.0,4.91,4.96,4.98,4.96,0.83
4,4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,...,1.0,1.0,365,no disponible todo el año,11.0,4.91,5.00,4.73,4.91,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26577,26577,26577,https://www.airbnb.com/rooms/1252141718111439700,2024-09-25,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,"Welcome to Lomas de Bezares, a neighborhood kn...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/67472234,HostPal,...,2.0,1.0,45,disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
26578,26578,26578,https://www.airbnb.com/rooms/1252159821531694005,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
26579,26579,26579,https://www.airbnb.com/rooms/1252185591721328995,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
26580,26580,26580,https://www.airbnb.com/rooms/1252218347521498445,2024-09-25,city scrape,Suite Queen | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70


_accommodates_

In [72]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['accommodates'])
unico

array([0., 1.])

In [69]:
df['accommodates'] = df['accommodates'].replace([1, 2, 2.9], 0).replace([3, 4, 5, 6, 7], 1)
df

,Unnamed: 0.1,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,...,1.0,1.0,7,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
1,1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,...,1.0,1.0,365,no disponible todo el año,64.0,4.58,4.70,4.78,4.98,0.41
2,2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,...,2.0,2.0,30,no disponible todo el año,49.0,4.90,4.75,4.92,4.98,0.31
3,3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,...,1.0,1.6,180,no disponible todo el año,127.0,4.91,4.96,4.98,4.96,0.83
4,4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,...,1.0,1.0,365,no disponible todo el año,11.0,4.91,5.00,4.73,4.91,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26577,26577,26577,https://www.airbnb.com/rooms/1252141718111439700,2024-09-25,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,"Welcome to Lomas de Bezares, a neighborhood kn...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/67472234,HostPal,...,2.0,1.0,45,disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
26578,26578,26578,https://www.airbnb.com/rooms/1252159821531694005,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
26579,26579,26579,https://www.airbnb.com/rooms/1252185591721328995,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70
26580,26580,26580,https://www.airbnb.com/rooms/1252218347521498445,2024-09-25,city scrape,Suite Queen | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,...,1.0,1.0,365,no disponible todo el año,0.0,4.80,4.90,4.90,4.90,1.70


In [71]:
df['availability_365']

0        no disponible todo el año
1        no disponible todo el año
2        no disponible todo el año
3        no disponible todo el año
4        no disponible todo el año
                   ...            
26577       disponible todo el año
26578    no disponible todo el año
26579    no disponible todo el año
26580    no disponible todo el año
26581    no disponible todo el año
Name: availability_365, Length: 26582, dtype: object

**REGRESIÓN LOGÍSTICA**

In [ ]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['pclass', 'age', 'fare']]
Var_Dep = df['host_is_superhost']